In [3]:
import pyomo.environ as pyo

In [40]:
model = pyo.ConcreteModel()

model = pyo.ConcreteModel()

model.llamas = pyo.Var(within=pyo.NonNegativeReals)
model.goats = pyo.Var(within=pyo.NonNegativeReals)

model.maximizeProfits = pyo.Objective(expr=200 * model.llamas  + 300 * model.goats, sense=pyo.maximize)

model.LaborConstraint = pyo.Constraint(expr = 3 * model.llamas + 2 * model.goats <= 100)
model.MedialConstraint = pyo.Constraint(expr=2* model.llamas + 4 * model.goats <= 120)
model.LandConstraint  = pyo.Constraint(expr = model.llamas + model.goats <= 45)
#model.maximizeNumberOfAnimals = pyo.Objective(expr=model.llamas + model.goats, sense=pyo.maximize)

optimizer = pyo.SolverFactory('glpk')
optimizer.solve(model)
print(model.display())

Model unknown

  Variables:
    llamas : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  20.0 :  None : False : False : NonNegativeReals
    goats : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  20.0 :  None : False : False : NonNegativeReals

  Objectives:
    maximizeProfits : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 10000.0

  Constraints:
    LaborConstraint : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 100.0 : 100.0
    MedialConstraint : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 120.0 : 120.0
    LandConstraint : Size=1
        Key  : Lower : Body : Upper
        None :  None : 40.0 :  45.0
None
